In [4]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("D:\OneDrive - NITT\Custom_Download\Abstimmung_1433_3_monts.csv",
                 delimiter=';',encoding='ANSI')
shop_number = df[df['KST']!=0]['KST'].unique()

In [5]:
df.shape

(1595, 8)

In [6]:
len(shop_number)

27

In [8]:
shop = 3
data = df[df['KST'].isin([shop]) | df['KST2'].isin([shop])]
data['Soll'] = np.where(data['KST']==shop,data['BETRAG'],np.nan)
data['Haben'] = np.where(data['KST2']==shop,data['BETRAG'],np.nan)

In [9]:
data 
# soll -> should
# haben-> have

,DATE,KTO,KST,GKTO,KST2,BETRAG,TEXT,SH,Soll,Haben
234,14.04.2022,1499,3,1503,0,3700.0,Filiale = 103,S,3700.0,NaN
250,14.04.2022,1800,0,1499,3,3700.0,Filialeinzahlung Fil 103,S,NaN,3700.0
1111,02.06.2022,1800,0,1499,3,5900.0,Filialeinzahlung Fil 103,S,NaN,5900.0
1115,03.06.2022,1499,3,1503,0,5900.0,Filiale = 103,S,5900.0,NaN
1475,24.06.2022,1499,3,1503,0,3600.0,Filiale = 103,S,3600.0,NaN
1485,24.06.2022,1800,0,1499,3,3600.0,Filialeinzahlung Fil 103,S,NaN,3600.0


In [4]:
def get_index(df,date,promised_amount,no_loopup_days=5):
    payment = df[['DATE','Haben']]
    offset = date + datetime.timedelta(days=no_loopup_days)
    
    small_df = payment[(payment['DATE']<= offset) & (payment['DATE']>= date)]
    small_df = small_df[~small_df['Haben'].isnull()]
    
    y = small_df[small_df['Haben']==promised_amount]
    indx = y.index[0] if len(y)>=1 else None
    return indx

In [5]:
pair_dict = dict()
for shop in sorted(shop_number):
    pair_list = []
    data = df[df['KST'].isin([shop]) | df['KST2'].isin([shop])]
    data['Soll'] = np.where(data['KST']==shop,data['BETRAG'],np.nan)
    data['Haben'] = np.where(data['KST2']==shop,data['BETRAG'],np.nan)

    data = data[['DATE','KST','KST2','Soll','Haben']]

    data['DATE'] = pd.to_datetime(data['DATE'],dayfirst=True,infer_datetime_format=True)

    data_clean = data[['DATE','Soll']][~data['Soll'].isnull()]
    for indx, row in data_clean.iterrows():
        indx_payment = get_index(data,row['DATE'],row['Soll'])
        Search_indx = sum(pair_list,())
        
        if row['Soll']<0:
            indx_payment = data_clean[(data_clean['DATE']== row['DATE']) & (data_clean['Soll']== abs(row['Soll']))].index[0]
            
            pair_list.append((indx_payment,indx)) 
        else:
            if indx_payment!=None:
                if indx_payment not in Search_indx:
                    pair_list.append((indx,indx_payment))
                    
    pair_dict[shop] = pair_list
                
indx_drop = sorted(sum(sum(pair_dict.values(),[]),()))

In [6]:
df_final = df.drop(indx_drop)
df_final.to_csv('Abstimmung_1433_3_monts_unreconciled.csv',index=False)

# NEW code

In [23]:
writer = pd.ExcelWriter('unreconciled.xlsx', engine='xlsxwriter')

for shop in sorted(shop_number):
    shop_data = df_final[df_final['KST'].isin([shop]) | df_final['KST2'].isin([shop])]
    if len(shop_data)>=1:
        shop_data.to_excel(writer, sheet_name=str(shop),startcol=0,index=False)
writer.save()

In [7]:
df.shape

(1595, 8)

In [8]:
len(df_final) + len(indx_drop)

1595

In [9]:
len(indx_drop)

1064

In [10]:
len(set(indx_drop))

1064

# TEST

In [11]:
import pandas as pd
pd.set_option('display.max_rows', 1700)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [12]:
pair_dict[11]

[(31, 65),
 (74, 112),
 (146, 172),
 (203, 230),
 (263, 290),
 (306, 340),
 (395, 418),
 (442, 516),
 (554, 575),
 (568, 596),
 (610, 646),
 (701, 727),
 (743, 776),
 (766, 789),
 (805, 838),
 (893, 930),
 (943, 966),
 (961, 991),
 (1007, 1048),
 (1071, 1088),
 (1158, 1162),
 (1193, 1218),
 (1277, 1297),
 (1390, 1410),
 (1428, 1468),
 (1483, 1509),
 (1522, 1545),
 (1563, 1590)]

In [13]:
shop = 11
df[df['KST'].isin([shop]) | df['KST2'].isin([shop])]

,DATE,KTO,KST,GKTO,KST2,BETRAG,TEXT,SH
12,01.04.2022,1800,0,1499,11,700.0,Filialeinzahlung Fil 411,S
13,01.04.2022,1800,0,1499,11,900.0,Filialeinzahlung Fil 411,S
31,02.04.2022,1499,11,1511,0,800.0,Filiale = 411,S
65,04.04.2022,1800,0,1499,11,800.0,Filialeinzahlung Fil 411,S
74,05.04.2022,1499,11,1511,0,800.0,Filiale = 411,S
112,06.04.2022,1800,0,1499,11,800.0,Filialeinzahlung Fil 411,S
146,08.04.2022,1499,11,1511,0,700.0,Filiale = 411,S
172,09.04.2022,1800,0,1499,11,700.0,Filialeinzahlung Fil 411,S
203,12.04.2022,1499,11,1511,0,900.0,Filiale = 411,S
230,13.04.2022,1800,0,1499,11,900.0,Filialeinzahlung Fil 411,S
